# Client Input Module

In [2]:
# Function to assess client risk profile
from clientprofile.client_risk_profile import client_risk_tolerance
client_risk_tolerance()

A. On a scale of 1(Risky Avoidant) - 4(Risky), how would your closest friend describe your risk tolerance? 4
B. On a scale of 1(Risky Avoidant) - 4(Risky), how would you describe your risk tolerance? 4
C. What would you rather:1. Take $1,000 in cash  2. A 50% chance at winning $5,000  3. A 25% chance at winning $10,000  4. A 5% chance at winning $100,000   4
D. Define risk: 1. Loss 2. Uncertainty 3. Opportunity 4. Thrill  4
E. How comfortable on a scale of 1(Not Comfortable) - 4(Very Comfortable)are you with investing your money into stocks? 4


1.0

# Risk Parity Model

In [ ]:
# Import modules.
import os
import pandas as pd
import numpy.random as rnd
from dotenv import load_dotenv
import requests
import warnings
import json
import sqlalchemy as sql
from pathlib import Path
import datetime
import yfinance as yf
warnings.filterwarnings('ignore')

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.4%}'.format

# Date range
today = datetime.date.today()
start_date = today - datetime.timedelta(days=252*5) #trading days(252) * amount of years to go back 

start = start_date.strftime("%Y-%m-%d")
end = today.strftime("%Y-%m-%d")

# Tickers of assets
assets = ["XLC", "XLY", "XLP", "XLE", "XLF", "XLV", "XLI", "XLB", "XLRE", "XLK", "XLU", "AAPL", "MSFT", "NVDA", "AMZN", "BRK-B",
         "GOOG", "META", "UNH", "XOM", "AGG", "BND", "LQD", "VCIT", "BNDX", "TMF", "TLT", "ICVT", "LKOR", "FBND"]
assets.sort()

# Downloading data
og_data = yf.download(assets, start = start, end = end)
data = og_data.loc[:,('Adj Close', slice(None))]
data.columns = assets

Y = data[assets].pct_change().dropna()

display(start)
display(end)
display(Y)

In [ ]:
x = float(input())

if (x >= 0) and (x < 0.25):
    fear_factor = 10000
    print(fear_factor)

elif (x >= 0.25) and (x < 0.5):
        fear_factor = 3500
        print(fear_factor)

elif (x >= 0.5) and (x <= 0.75):
        fear_factor = 2.5
        print(fear_factor)
else:
    if (x >= 0.75) and (x <= 1):
        fear_factor = 0
        print(fear_factor)
    else:
        print(f"Number isn't between 0 and 1")
       

In [ ]:
import riskfolio as rp

# Building the portfolio object
port = rp.Portfolio(returns=Y)

# Calculating optimal portfolio

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov, d=0.94)

# Estimate optimal portfolio:

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'Utility' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = fear_factor # Risk aversion factor, only useful when obj is 'Utility'

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

In [ ]:
print(fear_factor)

In [ ]:
# Plotting the composition of the portfolio

ax = rp.plot_pie(w=w, title='Risk Parity', others=0.05, nrow=25, cmap = "tab20",
                 height=6, width=10, ax=None)

In [ ]:
# Plotting the risk composition of the portfolio

ax = rp.plot_risk_con(w, cov=port.cov, returns=port.returns, rm=rm, rf=0, alpha=0.01,
                      color="tab:blue", height=6, width=10, ax=None)

In [ ]:
w.to_csv(Path("w.csv"))